<a href="https://colab.research.google.com/github/zakariaabou/introduction-to-github/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t',names = [1,2])
test_data = pd.read_csv(test_file_path, sep='\t',names = [1,2])
y_train = train_data.pop(1)
y_train = y_train.replace(['ham','spam'],[0,1]).values
y_test = test_data.pop(1)
y_test = y_test.replace(['ham','spam'],[0,1]).values

In [ ]:
max_features = 10000  # Maximum number of words to keep
sequence_length = 100  # Length of each input sequence

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

vectorize_layer.adapt(train_data[2].values)
X_train = vectorize_layer(train_data[2].values)
X_test = vectorize_layer(test_data[2].values)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=max_features, output_dim=16, input_length=sequence_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.5),  # Add dropout layer to prevent overfitting
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.5),  # Add another dropout layer
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Preprocess the input message
  pred_text = tf.expand_dims(pred_text, 0)  # Add a batch dimension
  pred_text = vectorize_layer(pred_text)    # Vectorize

  # Predict probability of spam
  x = model.predict(pred_text)[0][0]

  # Determine the label based on probability
  label = "spam" if x > 0.5 else "ham"

  prediction = [x, label]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True


  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
